## Data Science - Group 15

### Topic: Toronto E-Bike demographic, travel and public opinion data

In [88]:
##### Data Ingestion

import pandas as pd

raw_data_path = "../data/E-Bike_Survey_Responses.csv"
columnNames = names=['TimeStamp','AgeRange','Sex','PhysicalHealth','Education','HouseholdIncome','EmploymentStatus','District','AvgDistanceTravel','AvgCommute','PreferedTransportation','PrivateMotorizedVehicles','SupportingStatements','MultiUseTrails','AwarePathSpeedLimit','WitnessedCollisionConflict','OpinionSpeedLimit','OpinionPathBylaw','BicycleLaneUsage','OpinionLaneBylaw','OpinionSidewalk','OpinionPersonalMobilityDevice'] # names of columns

survey = pd.read_csv(raw_data_path, names=columnNames, skiprows=1)

survey.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2238 entries, 0 to 2237
Data columns (total 22 columns):
TimeStamp                        2238 non-null object
AgeRange                         2234 non-null object
Sex                              2221 non-null object
PhysicalHealth                   2227 non-null object
Education                        2220 non-null object
HouseholdIncome                  2178 non-null object
EmploymentStatus                 2223 non-null object
District                         2237 non-null object
AvgDistanceTravel                2237 non-null object
AvgCommute                       2237 non-null object
PreferedTransportation           2238 non-null object
PrivateMotorizedVehicles         2238 non-null object
SupportingStatements             2238 non-null object
MultiUseTrails                   2238 non-null object
AwarePathSpeedLimit              2238 non-null object
WitnessedCollisionConflict       2238 non-null object
OpinionSpeedLimit            

In [89]:
survey = survey.dropna()
survey.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2138 entries, 0 to 2237
Data columns (total 22 columns):
TimeStamp                        2138 non-null object
AgeRange                         2138 non-null object
Sex                              2138 non-null object
PhysicalHealth                   2138 non-null object
Education                        2138 non-null object
HouseholdIncome                  2138 non-null object
EmploymentStatus                 2138 non-null object
District                         2138 non-null object
AvgDistanceTravel                2138 non-null object
AvgCommute                       2138 non-null object
PreferedTransportation           2138 non-null object
PrivateMotorizedVehicles         2138 non-null object
SupportingStatements             2138 non-null object
MultiUseTrails                   2138 non-null object
AwarePathSpeedLimit              2138 non-null object
WitnessedCollisionConflict       2138 non-null object
OpinionSpeedLimit            

In [90]:
#Drop TimeStamp Column, it does not provide any useful information
survey = survey.drop('TimeStamp', axis = 1)

In [91]:
#print(survey['Sex'].value_counts()) #Light Cleaning needed - some non-standard/open ended replies 
#print(survey['AgeRange'].value_counts()) # Is Clean
#print(survey['PhysicalHealth'].value_counts()) #Light Cleaning needed - some non-standard/open ended replies 
#print(survey['Education'].value_counts()) #Light Cleaning needed - some non-standard/open ended replies
#print(survey['HouseholdIncome'].value_counts()) #Is Clean
#print(survey['EmploymentStatus'].value_counts()) #Light Cleaning needed - some non-standard/open ended replies 
#print(survey['District'].value_counts()) #Heavy cleaning needed - alot of non-standard/open ended replies
#print(survey['AvgDistanceTravel'].value_counts())#Is Clean
#print(survey['AvgCommute'].value_counts())#Is Clean
#print(survey['PreferedTransportation'].value_counts())#Light Cleaning needed - some non-standard/open ended responces 
#print(survey['PrivateMotorizedVehicles'].value_counts())#Heavy cleaning needed - alot of non-standard/open ended responces
#print(survey['SupportingStatements'].value_counts()) #Moderate Cleaning needed - various combinations of responces, all appear to be bound
#print(survey['MultiUseTrails'].value_counts()) #Moderate Cleaning needed - various combinations of responces, all appear to be bound
#print(survey['AwarePathSpeedLimit'].value_counts())# Is Clean
#print(survey['WitnessedCollisionConflict'].value_counts())#Moderate Cleaning needed - various combinations of responces, all appear to be bound
#print(survey['OpinionSpeedLimit'].value_counts())#Heavy Cleaning needed - various combinations of responces and openeded responces
#print(survey['OpinionPathBylaw'].value_counts())  #Moderate Cleaning needed - various combinations of responces and openeded responces        
#print(survey['BicycleLaneUsage'].value_counts()) #Moderate Cleaning needed - various combinations of responces and openeded responces       
#print(survey['OpinionLaneBylaw'].value_counts()) #Heavy Cleaning needed - various combinations of responces and openeded responces
#print(survey['OpinionSidewalk'].value_counts()) #Moderate Cleaning needed - various combinations of responces and openeded responces           
#print(survey['OpinionPersonalMobilityDevice'].value_counts()) #Moderate Cleaning needed - various combinations of responces and openeded responces                     
               

#As I see it our options (or combination of options) to clean non-standard/open ended responces:
# 1) Drop rows
# 2) Group non-standard/open ended responces it's own category
# 3) Try to fit non-standard/open ended responces into existing categories

#my intial thought it #2 for majority features

#For the data that has various combinations, i.e. those that appear as if they were picked from a picker list, we need to identify all the unique options and: 
#1) do some sort of multi-index
#2) create new Categorical columns - Yes and No for each unique option

#my intial thought it #2 for majority of the feature



In [92]:
#Useful Function

#Function that retruns unique list from a given list
def unique_list(lst):
    unique_list = []
    for x in lst:
        if x not in unique_list:
            unique_list.append(x)
    return unique_list

#Function that retruns datafame with all values other then 'keepValues' in 'feature' replaced with 'replaceWith' in Dataframe 'df'
def Replace_All_Other_Values(df, feature,keepValues,replaceWith):
    for item in df[feature]:
        if item in keepValues:
            pass
        else:
            df = df.replace({feature: item},replaceWith)
    return df


    

In [97]:
#Clean Sex feature
survey = Replace_All_Other_Values(survey, 'Sex',['Male','Female'], 'Other')
#survey['Sex'].value_counts()

#Clean PhysicalHealth feature
survey = Replace_All_Other_Values(survey, 'PhysicalHealth',['Good','Excellent','Very good','Fairly good','Poor'], 'Other')
#survey['PhysicalHealth'].value_counts()

#Clean Education feature
survey = Replace_All_Other_Values(survey, 'Education',['University degree','Post graduate','College or trade school diploma','High school diploma '], 'Other')
#survey['Education'].value_counts()

#Clean EmploymentStatus feature
survey = Replace_All_Other_Values(survey, 'EmploymentStatus',['Full Time','Self Employed','Retired','Part Time','Student','Unemployed','Home Maker'], 'Other')
#survey['EmploymentStatus'].value_counts()            

#Clean District feature
survey = Replace_All_Other_Values(survey, 'District',['Central Toronto, York or East York','Etobicoke','North York','Scarborough','Mississauga','Brampton'], 'Other')
#survey['District'].value_counts()  

#Clean PreferedTransportation feature
survey = Replace_All_Other_Values(survey, 'PreferedTransportation',['bicycle','Etobicoke','private motor vehicle (car, truck, SUV, van, motorcycle, gas limited speed motorcycle or moped)','transit','walking','scooter type e-bike','pedal assist type e-bike'], 'Other')
#survey['PreferedTransportation'].value_counts()

#Clean PreferedTransportation feature
survey = Replace_All_Other_Values(survey, 'PreferedTransportation',['bicycle','Etobicoke','private motor vehicle (car, truck, SUV, van, motorcycle, gas limited speed motorcycle or moped)','transit','walking','scooter type e-bike','pedal assist type e-bike'], 'Other')
#survey['PreferedTransportation'].value_counts()


#Clean PrivateMotorizedVehicles feature
#for item in survey['PrivateMotorizedVehicles']:
#        if "car" in item.lower() or "suv" in item.lower() or "truck" in item.lower() or "van" in item.lower():
#            survey['HasCarSUVTruckOrVan'] = survey.replace({'PrivateMotorizedVehicles': item}, "Car, SUV, Truck or Van")
#        elif item == "No - I do not have access to a private motorized vehicle":
#            survey = survey.replace({'PrivateMotorizedVehicles': item}, "No private motorized vehicle")
#            pass
#        elif "motorcycle" in item.lower():
#            survey = survey.replace({'PrivateMotorizedVehicles': item}, "Motorcycle")
#        elif "bicycle" in item.lower():
#            survey = survey.replace({'PrivateMotorizedVehicles': item}, "Bicycle")
#        elif "bike" in item.lower() and ("pedal" in item.lower() or "assist" in item.lower()):
#            survey = survey.replace({'PrivateMotorizedVehicles': item}, "Pedal Assist E-bike") 
#        elif ("bike" in item.lower() and "scooter" in item.lower()): 
#            survey = survey.replace({'PrivateMotorizedVehicles': item}, "Scooter E-bike")             
#        elif ("scooter" in item.lower() and not "bike" in item.lower()) or "moped" in item.lower() or "vespa" in item.lower():
#            survey = survey.replace({'PrivateMotorizedVehicles': item}, "Gas E-bike") 
#        elif "share" in item.lower() or "zip" in item.lower() or "rental" in item.lower():
#            survey = survey.replace({'PrivateMotorizedVehicles': item}, "AutoShare") 

#Clean PrivateMotorizedVehicles feature
#for item in survey['PrivateMotorizedVehicles']:

survey['HasCarSUVTruckOrVan'] = "No"
survey['HasMotorcycle'] = "No"
survey['HasBicycle'] = "No"
survey['HasPAEbike'] = "No"
survey['HasScooterEbike'] = "No" 
survey['HasGasEbike'] = "No" 
survey['HasAutoShare'] = "No"

for item, row in survey.iterrows():
        if "car" in row['PrivateMotorizedVehicles'].lower() or "suv" in row['PrivateMotorizedVehicles'].lower() or "truck" in row['PrivateMotorizedVehicles'].lower() or "van" in row['PrivateMotorizedVehicles'].lower():
            survey.loc[[item],['HasCarSUVTruckOrVan']] = "Yes"
        if "motorcycle" in row['PrivateMotorizedVehicles'].lower():
            survey.loc[[item],['HasMotorcycle']] = "Yes"
        if "bicycle" in row['PrivateMotorizedVehicles'].lower():
            survey.loc[[item],['HasBicycle']] = "Yes"
        if "bike" in row['PrivateMotorizedVehicles'].lower() and ("pedal" in row['PrivateMotorizedVehicles'].lower() or "assist" in row['PrivateMotorizedVehicles'].lower()):
            survey.loc[[item],['HasPAEbike']] = "Yes"
        if ("bike" in row['PrivateMotorizedVehicles'].lower() and "scooter" in row['PrivateMotorizedVehicles'].lower()): 
            survey.loc[[item],['HasScooterEbike']] = "Yes"             
        if ("scooter" in row['PrivateMotorizedVehicles'].lower() and not "bike" in row['PrivateMotorizedVehicles'].lower()) or "moped" in row['PrivateMotorizedVehicles'].lower() or "vespa" in row['PrivateMotorizedVehicles'].lower():
            survey.loc[[item],['HasGasEbike']] = "Yes"  
        if "share" in row['PrivateMotorizedVehicles'].lower() or "zip" in row['PrivateMotorizedVehicles'].lower() or "rental" in row['PrivateMotorizedVehicles'].lower():
            survey.loc[[item],['HasAutoShare']] = "Yes"
          
survey[['PrivateMotorizedVehicles','HasCarSUVTruckOrVan','HasMotorcycle','HasBicycle','HasPAEbike','HasScooterEbike','HasGasEbike','HasAutoShare']]
        
#survey['PrivateMotorizedVehicles'].value_counts()


# break statments by ',' - final all uqunie statments - create new feature for statements
#survey['SupportingStatements'].value_counts()




,PrivateMotorizedVehicles,HasCarSUVTruckOrVan,HasMotorcycle,HasBicycle,HasPAEbike,HasScooterEbike,HasGasEbike,HasAutoShare
0,Yes - a scooter style e-bike,No,No,No,No,Yes,No,No
1,Yes - a scooter style e-bike,No,No,No,No,Yes,No,No
2,"Yes - a car, SUV, truck or van",Yes,No,No,No,No,No,No
3,"Yes - a car, SUV, truck or van",Yes,No,No,No,No,No,No
4,"Yes - a car, SUV, truck or van",Yes,No,No,No,No,No,No
5,No - I do not have access to a private motoriz...,No,No,No,No,No,No,No
6,"Yes, a motorcycle",No,Yes,No,No,No,No,No
7,"Yes - a car, SUV, truck or van",Yes,No,No,No,No,No,No
8,"Yes - a car, SUV, truck or van",Yes,No,No,No,No,No,No
9,"Yes - a car, SUV, truck or van",Yes,No,No,No,No,No,No
